In [2]:
# Import utility.py and db_utility
import scripts.utils as ut
from scripts.db_utils import get_db_engine, open_connection, close_connection, get_table_count
from sqlalchemy import text

# Load full dataset with all city characteristics [cleaned and standardized columns]
df = ut.read_world_colCleanDataset()

print(df.head())
print(f"✅ Loaded dataset with {len(df)} rows and {len(df.columns)} columns")

   id      city      country  meal,_inexpensive_restaurant_(usd)  \
0   0     Delhi        India                                4.90   
1   1  Shanghai        China                                5.59   
2   2   Jakarta    Indonesia                                2.54   
3   3    Manila  Philippines                                3.54   
4   4     Seoul  South Korea                                7.16   

   meal_for_2_people,_mid-range_restaurant,_three-course_(usd)  \
0                                              22.04             
1                                              40.51             
2                                              22.25             
3                                              27.40             
4                                              52.77             

   mcmeal_at_mcdonalds_(or_equivalent_combo_meal)_(usd)  \
0                                               4.28      
1                                               5.59      
2                

In [3]:
# -- Clean and prepare dataset

# Remove any rows with missing city or country
df = df.dropna(subset=['city', 'country'])

# Keep only rows with data_quality == 1 (reliable data)
df = df[df['data_quality'] == 1]

# Reset index for iteration
df = df.reset_index(drop=True)

print(f"✅ Dataset cleaned: {len(df)} rows ready for processing")

✅ Dataset cleaned: 910 rows ready for processing


In [4]:
# -- DB connection & reference tables
engine = get_db_engine()
conn = open_connection(engine)

# Fetch regcar
regcar_df = conn.execute(text("SELECT idcara, descar FROM regcar")).fetchall()
regcar_df = ut.pd.DataFrame(regcar_df, columns=['idcara', 'descar'])

# Fetch regcit
regcit_df = conn.execute(text("SELECT idncity, namecit, country FROM regcit")).fetchall()
regcit_df = ut.pd.DataFrame(regcit_df, columns=['idncity', 'namecit', 'country'])


# -- Test: check selected columns match regcar descriptions
# Columns from dataset that we plan to map and from db
dataset_columns = df.columns.tolist()
car_to_use = regcar_df['descar'].tolist()

# Fetch columns
mapped_cols = [col for col in dataset_columns if col in car_to_use]

# print("\nColumns matched/mapped:", mapped_cols)
print(f"✅ Total columns to insert into valcar: {len(mapped_cols)}")

print(f"✅ Fetched {len(regcar_df)} characteristics and {len(regcit_df)} cities")


🔍 Loading .env
🔧 Variables loaded: user=user, host=localhost, db=livingcost
✅ Total columns to insert into valcar: 44
✅ Fetched 44 characteristics and 4874 cities


In [5]:
# -- Prepare mapping for valcar insertion

# Create a dictionary: column name -> idcara
col_to_idcara = dict(zip(regcar_df['descar'], regcar_df['idcara']))

# Create a dictionary: (city, country) -> idncity
city_country_to_idncity = {
    (row['namecit'], row['country']): row['idncity']
    for idx, row in regcit_df.iterrows()
}

# -- Test
# print(city_country_to_idncity)
# print(col_to_idcara)
print(f"✅ Mapped {len(col_to_idcara)} characteristics and {len(city_country_to_idncity)} cities")


✅ Mapped 44 characteristics and 4874 cities


In [6]:
# -- Insert data into valcar

inserted = 0
skipped = 0

# Open DB connection
engine = get_db_engine()
conn = open_connection(engine)

# Iterate over dataset rows
for idx, row in df.iterrows():
    city = row['city'].strip()
    country = row['country'].strip()

    # Get city id
    idncity = city_country_to_idncity.get((city, country))
    if not idncity:
        print(f"⚠️ City '{city}' in '{country}' not found in regcit, skipping row {idx}")
        skipped += len(col_to_idcara)
        continue

    # Iterate over characteristics columns in dataset
    for col_name, idcara in col_to_idcara.items():
        cvalue = row.get(col_name)

        if cvalue is None or (isinstance(cvalue, float) and ut.np.isnan(cvalue)):
            continue  # skip empty values

        # Check if the record already exists
        exists = conn.execute(
            text("""
                SELECT 1 FROM valcar 
                WHERE idncity = :idncity AND idcara = :idcara
            """),
            {"idncity": idncity, "idcara": idcara}
        ).fetchone()

        if exists:
            skipped += 1
            continue

        # Insert into valcar
        conn.execute(
            text("""
                INSERT INTO valcar (idncity, idcara, cvalue)
                VALUES (:idncity, :idcara, :cvalue)
            """),
            {"idncity": idncity, "idcara": idcara, "cvalue": str(cvalue)}
        )
        inserted += 1

# Commit changes and close connection
conn.commit()
close_connection(conn)

print(f"✅ valcar populated: {inserted} inserted, {skipped} skipped")

🔍 Loading .env
🔧 Variables loaded: user=user, host=localhost, db=livingcost
✅ DB connection closed
✅ valcar populated: 39916 inserted, 0 skipped
